**DATASET GENERATION**

In [ ]:
# Environment and versions capture (keeps Colab and server in sync)
import sys, json, platform
import sklearn, numpy as np, scipy, pandas as pd, joblib

versions = {
    "python": sys.version.split()[0],
    "platform": platform.platform(),
    "sklearn": sklearn.__version__,
    "numpy": np.__version__,
    "scipy": scipy.__version__,
    "pandas": pd.__version__,
    "joblib": joblib.__version__,
}
print("Versions:", versions)
with open('colab_versions.json', 'w') as f:
    json.dump(versions, f, indent=2)
print("Saved to colab_versions.json")

# Optional: warn if versions differ from server pins
expected = {"sklearn": "1.6.1", "numpy": "2.1.3", "scipy": "1.14.1", "pandas": "2.2.3", "joblib": "1.4.2"}
mismatches = []
for k, v in expected.items():
    have = versions.get(k)
    if have and have != v:
        mismatches.append(f"{k}: have {have}, want {v}")

if mismatches:
    print("Compatibility warning (server expects pinned versions):")
    for m in mismatches:
        print(" -", m)
else:
    print("Environment matches server pins.")

In [ ]:
# ========================================
# FurEver.AI - Data Generation (Colab-ready)
# ========================================

import pandas as pd
import numpy as np

# Reproducibility
np.random.seed(42)

print("--- Starting Data Generation ---")

# =======================================================
# 1) Generate Pet Profiles (Items)
# =======================================================
num_pets = 1000

pet_data = {
    'Pet_ID': [f'P{i:04d}' for i in range(1, num_pets + 1)],
    'Type': np.random.choice(['Dog', 'Cat'], size=num_pets, p=[0.6, 0.4]),
    'Age_Group': np.random.choice(['Puppy/Kitten', 'Young_Adult', 'Senior'], size=num_pets, p=[0.3, 0.5, 0.2]),
    'Size': np.random.choice(['Small', 'Medium', 'Large'], size=num_pets, p=[0.35, 0.45, 0.2]),
    # Energy Level: 1=Low, 2=Medium, 3=High (Matches Adopter Activity)
    'Energy_Level': np.random.choice([1, 2, 3], size=num_pets, p=[0.3, 0.4, 0.3]),
    # Good_With_Kids: 0=No, 1=Yes, 2=Needs older kids (Matches Adopter Has_Kids)
    'Good_With_Kids': np.random.choice([0, 1, 2], size=num_pets, p=[0.1, 0.7, 0.2]),
    'Grooming_Needs': np.random.choice(['Low', 'Medium', 'High'], size=num_pets, p=[0.5, 0.3, 0.2]),
}

pet_profiles = pd.DataFrame(pet_data)
pet_profiles.to_csv('pet_profiles.csv', index=False)
print(f"Generated and saved pet_profiles.csv with {len(pet_profiles)} records.")

# =======================================================
# 2) Generate Adopter Profiles (Users)
# =======================================================
num_adopters = 500

adopter_data = {
    'Adopter_ID': [f'U{i:03d}' for i in range(1, num_adopters + 1)],
    'Housing_Type': np.random.choice(['Apartment', 'House_No_Yard', 'House_Yard'], size=num_adopters, p=[0.4, 0.3, 0.3]),
    'Has_Kids': np.random.choice([0, 1], size=num_adopters, p=[0.6, 0.4]),
    # Time_At_Home: 1=Away_Most_Day, 2=Hybrid, 3=WFH_Full_Time
    'Time_At_Home': np.random.choice([1, 2, 3], size=num_adopters, p=[0.25, 0.4, 0.35]),
    # Activity Level: 1=Low, 2=Medium, 3=High (Matches Pet Energy)
    'Activity_Level': np.random.choice([1, 2, 3], size=num_adopters, p=[0.3, 0.4, 0.3]),
    'Experience_Level': np.random.choice(['First_Time', 'Past_Owner', 'Expert'], size=num_adopters, p=[0.3, 0.5, 0.2]),
    'Pet_Type_Desired': np.random.choice(['Dog', 'Cat'], size=num_adopters, p=[0.7, 0.3]),
}

adopter_profiles = pd.DataFrame(adopter_data)
adopter_profiles.to_csv('adopter_profiles.csv', index=False)
print(f"Generated and saved adopter_profiles.csv with {len(adopter_profiles)} records.")

# =======================================================
# 3) Generate Historical Matches (Target for Matching)
# =======================================================
num_attempts = 3000
historical_matches = pd.DataFrame({
    'Adopter_ID': np.random.choice(adopter_profiles['Adopter_ID'], size=num_attempts),
    'Pet_ID': np.random.choice(pet_profiles['Pet_ID'], size=num_attempts),
})

# Merge to build rules
history = pd.merge(historical_matches, adopter_profiles, on='Adopter_ID')
history = pd.merge(history, pet_profiles, on='Pet_ID')

# Target
history['Success_Match'] = 0

# Rule 1: Energy alignment helps
history.loc[history['Activity_Level'] == history['Energy_Level'], 'Success_Match'] = 1

# Rule 2: Apartment + Large pet hurts
history.loc[(history['Housing_Type'] == 'Apartment') & (history['Size'] == 'Large'), 'Success_Match'] = 0

# Rule 3: Has kids but pet not good with kids hurts
history.loc[(history['Has_Kids'] == 1) & (history['Good_With_Kids'] == 0), 'Success_Match'] = 0

# Rule 4: Expert owner + High grooming needs helps
history.loc[(history['Experience_Level'] == 'Expert') & (history['Grooming_Needs'] == 'High'), 'Success_Match'] = 1

# Add noise
noise_count = int(num_attempts * 0.1)
history.loc[history.sample(noise_count, random_state=42).index, 'Success_Match'] = np.random.choice([0, 1], size=noise_count)

# Finalize training matches
historical_matches_final = history[['Adopter_ID', 'Pet_ID', 'Success_Match']].drop_duplicates()
historical_matches_final.to_csv('historical_matches.csv', index=False)
print(f"Generated and saved historical_matches.csv with {len(historical_matches_final)} records (The Training Data).")

# =======================================================
# 4) Prepare Training Datasets for 6 ML Algorithms
# =======================================================
print("--- Preparing Training Datasets for 6 ML Algorithms ---")

# ---------- 4.1 Decision Tree (Pawsonality) ----------
dt_data = adopter_profiles.copy()

def classify_pawsonality(row):
    # Active Adventurer: High activity & experienced
    if row['Activity_Level'] == 3 and row['Experience_Level'] in ['Past_Owner', 'Expert']:
        return 'Active Adventurer'
    # Cozy Companion: Low activity & Apartment
    elif row['Activity_Level'] == 1 and row['Housing_Type'] == 'Apartment':
        return 'Cozy Companion'
    # Playful Enthusiast: High activity & not always at home
    elif row['Activity_Level'] == 3 and row['Time_At_Home'] in [1, 2]:
        return 'Playful Enthusiast'
    # Confident Guardian: Expert & House with Yard
    elif row['Experience_Level'] == 'Expert' and row['Housing_Type'] == 'House_Yard':
        return 'Confident Guardian'
    # Gentle Nurturer: First/Past owner & WFH full-time
    elif row['Experience_Level'] in ['First_Time', 'Past_Owner'] and row['Time_At_Home'] == 3:
        return 'Gentle Nurturer'
    # Quiet Caretaker: Low activity & not always at home
    elif row['Activity_Level'] == 1 and row['Time_At_Home'] in [1, 2]:
        return 'Quiet Caretaker'
    # Social Butterfly: Medium activity & has kids
    elif row['Activity_Level'] == 2 and row['Has_Kids'] == 1:
        return 'Social Butterfly'
    # Balanced Buddy: default
    else:
        return 'Balanced Buddy'

dt_data['Target_Pawsonality'] = dt_data.apply(classify_pawsonality, axis=1)

dt_training_data = dt_data[['Adopter_ID', 'Housing_Type', 'Has_Kids', 'Time_At_Home', 'Activity_Level', 'Experience_Level', 'Target_Pawsonality']]
dt_training_data.to_csv('1_dt_pawsonality_training.csv', index=False)
print("1. Decision Tree (Pawsonality) Training Data saved.")

# ---------- 4.2 + 4.5 SVM & ANN (Core/Deep Matching) ----------
# Activity_Energy_Diff and one-hot categorical columns
match_df = pd.merge(historical_matches_final, adopter_profiles, on='Adopter_ID')
match_df = pd.merge(match_df, pet_profiles, on='Pet_ID')
match_df['Activity_Energy_Diff'] = np.abs(match_df['Activity_Level'] - match_df['Energy_Level'])

match_features = match_df[['Adopter_ID', 'Pet_ID', 'Activity_Energy_Diff', 'Has_Kids', 'Good_With_Kids', 'Experience_Level', 'Grooming_Needs', 'Size', 'Success_Match']]
svm_ann_training_data = pd.get_dummies(match_features, columns=['Experience_Level', 'Grooming_Needs', 'Size'], drop_first=True)
svm_ann_training_data = svm_ann_training_data.drop(columns=['Adopter_ID', 'Pet_ID'])

svm_ann_training_data.to_csv('2_5_svm_ann_matching_training.csv', index=False)
print("2 & 5. SVM/ANN (Matching) Training Data saved.")

# ---------- 4.3 Naive Bayes (Auto-Tagging with better rules) ----------
nb_data = pet_profiles.copy()

def generate_description(row):
    # Base description
    desc = f"A {row['Age_Group'].lower().replace('_', ' ')} {row['Type'].lower()} with {row['Grooming_Needs'].lower()} grooming needs. "
    tags = []

    # Age-specific tags and text
    if row['Age_Group'] == 'Senior':
        desc += "Calm and wise, ideal for a relaxed home. "
        tags.append('senior')
    elif row['Age_Group'] == 'Young_Adult':
        desc += "In the prime age for bonding and training. "
        tags.append('young-adult')
    elif row['Age_Group'] == 'Puppy/Kitten':
        if row['Type'] == 'Dog':
            desc += "Young and learning, needs patience and training. "
            tags.append('puppy')
        else:
            desc += "Playful and curious, perfect for a nurturing home. "
            tags.append('kitten')

    # Energy and living situation
    if row['Energy_Level'] == 3:
        desc += "Requires lots of space and playtime; thrives with an active lifestyle. "
        tags.extend(['high-energy'])
        # For large + high-energy we often prefer yard
        if row['Size'] == 'Large':
            tags.append('house-with-yard')
    elif row['Energy_Level'] == 1:
        desc += "Very calm and quiet, perfect for an apartment-approved life. "
        tags.extend(['low-energy', 'apartment-approved'])

    # Family/kids compatibility
    if row['Good_With_Kids'] == 1:
        desc += "Loves kids and other pets. "
        tags.append('family-friendly')
        if row['Energy_Level'] >= 2:
            tags.append('social')
    elif row['Good_With_Kids'] == 2:
        desc += "Best with experienced families and older children. "
        tags.append('experienced-owner-recommended')
    else:
        desc += "Best suited for a quiet, adult-only household. "
        tags.append('adults-only')

    # Size-based nuance
    if row['Size'] == 'Large':
        desc += "Due to size, a house with a yard is recommended. "
        tags.append('large-breed')

    # De-duplicate and return
    tags = list(dict.fromkeys(tags))
    return desc.strip(), tags

nb_data[['Pet_Description', 'Target_Pet_Tags']] = nb_data.apply(
    lambda row: pd.Series(generate_description(row)), axis=1
)

nb_training_data = nb_data[['Pet_ID', 'Pet_Description', 'Target_Pet_Tags']]
nb_training_data.to_csv('3_nb_autotagging_training.csv', index=False)
print("3. Naive Bayes (Auto-Tagging) Training Data saved.")

# ---------- 4.4 KNN (Recommendations) ----------
# Keep compatibility with earlier training code that expected duplicated names like 'Pet_Medium.1'
knn_data = pet_profiles.copy()
knn_training_data = pd.get_dummies(
    knn_data.drop(columns=['Pet_ID']),
    columns=['Type', 'Age_Group', 'Size', 'Grooming_Needs'],
    prefix='Pet'  # Intentional: may create duplicates like Pet_Medium (size) vs Pet_Medium.1 (grooming)
)
# Drop Good_With_Kids for similarity (optional, matches earlier approach)
if 'Good_With_Kids' in knn_training_data.columns:
    knn_training_data = knn_training_data.drop(columns=['Good_With_Kids'])
# Index by Pet_ID for lookup
knn_training_data.index = knn_data['Pet_ID']
knn_training_data.to_csv('4_knn_recommendation_features.csv')
print("4. KNN (Recommendations) Feature Data saved.")

# ---------- 4.6 Linear Regression (Adoption Timeline) ----------
lr_data = pet_profiles.copy()

def generate_days_in_shelter(row):
    base_days = 25
    if row['Energy_Level'] == 3: base_days += 10
    elif row['Energy_Level'] == 1: base_days -= 5

    if row['Size'] == 'Large': base_days += 15
    elif row['Size'] == 'Small': base_days -= 5

    if row['Age_Group'] in ['Senior', 'Puppy/Kitten']: base_days += 5
    elif row['Age_Group'] == 'Young_Adult': base_days -= 10

    if row['Grooming_Needs'] == 'High': base_days += 10

    noise = np.random.randint(-10, 10)
    return max(5, base_days + noise)

lr_data['Target_Days_In_Shelter'] = lr_data.apply(generate_days_in_shelter, axis=1)
# One-hot for LR training
lr_training = pd.get_dummies(
    lr_data.drop(columns=['Pet_ID']),
    columns=['Type', 'Age_Group', 'Size', 'Grooming_Needs'],
    drop_first=True
)
# Optional: remove Good_With_Kids (as earlier)
if 'Good_With_Kids' in lr_training.columns:
    lr_training = lr_training.drop(columns=['Good_With_Kids'])

lr_training.to_csv('6_lr_adoption_prediction_training.csv', index=False)
print("6. Linear Regression (Adoption Prediction) Training Data saved.")

print("--- Data Generation Complete ---")

--- Starting Data Generation ---
Generated and saved pet_profiles.csv with 1000 records.
Generated and saved adopter_profiles.csv with 500 records.
Generated and saved historical_matches.csv with 2988 records (The Training Data).
--- Preparing Training Datasets for 6 ML Algorithms ---
1. Decision Tree (Pawsonality) Training Data saved.
2 & 5. SVM/ANN (Matching) Training Data saved.
3. Naive Bayes (Auto-Tagging) Training Data saved.
4. KNN (Recommendations) Feature Data saved.
6. Linear Regression (Adoption Prediction) Training Data saved.
--- Data Generation Complete ---


**SVM**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Note: SVC stands for Support Vector Classifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import joblib # For saving the model
import numpy as np # Needed for predicting probabilities

print("--- Starting SVM Model Training (Core Matching) ---")

# 1. Load the Prepared Data
data = pd.read_csv('2_5_svm_ann_matching_training.csv')

# 2. Separate Features (X) and Target (y)
X = data.drop('Success_Match', axis=1)
y = data['Success_Match']

# 3. Split Data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"Training on {len(X_train)} records, Testing on {len(X_test)} records.")

# 4. Scale Features (Crucial for SVM)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print("Features scaled successfully.")

# 5. Train the Support Vector Machine Model
# *** CRITICAL CHANGE: probability=True is added here ***
# This enables the model to output a probability (score) instead of just a 0 or 1.
svm_model = SVC(kernel='linear', random_state=42, probability=True)
svm_model.fit(X_train_scaled, y_train)
print("SVM Model training complete.")

# 6. Evaluate the Model
# Use predict() for the traditional classification metrics
y_pred_class = svm_model.predict(X_test_scaled)

# Use predict_proba() to get the actual match score/probability
# probas[:, 1] takes the probability for the "Success" class (Class 1)
y_pred_probas = svm_model.predict_proba(X_test_scaled)
match_scores = np.max(y_pred_probas, axis=1) # The max probability (the confidence score)

print("\n--- Model Evaluation ---")
# Report based on the hard classification
print(f"Accuracy: {accuracy_score(y_test, y_pred_class):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_class))

# Report based on the required score output
print("\n--- Example Score Output Check ---")
print(f"Sample Predicted Match Scores (Probabilities): {match_scores[:5]}")
# Example: If the first prediction was 0.92, it would be the '92% match'
print(f"Sample score on first test record (Success Class): {y_pred_probas[0][1]:.2f}")


# 7. Save the trained model and the scaler (needed for deployment)
# The saved model now has the ability to calculate probabilities
joblib.dump(svm_model, 'furever_svm_model.pkl')
joblib.dump(scaler, 'furever_svm_scaler.pkl')
print("\nSVM Model (with probability enabled) and Scaler saved.")

--- Starting SVM Model Training (Core Matching) ---
Training on 2390 records, Testing on 598 records.
Features scaled successfully.
SVM Model training complete.

--- Model Evaluation ---
Accuracy: 0.9013

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.92      0.92       389
           1       0.86      0.86      0.86       209

    accuracy                           0.90       598
   macro avg       0.89      0.89      0.89       598
weighted avg       0.90      0.90      0.90       598


--- Example Score Output Check ---
Sample Predicted Match Scores (Probabilities): [0.85309886 0.80602751 0.99286248 0.992865   0.80594525]
Sample score on first test record (Success Class): 0.15

SVM Model (with probability enabled) and Scaler saved.


**DECISION TREES**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import joblib

print("--- Starting Decision Tree Model Training (Pawsonality) ---")

# 1. Load the Prepared Data
dt_data = pd.read_csv('1_dt_pawsonality_training.csv')

# Drop the Adopter_ID as it is not a feature
dt_data = dt_data.drop('Adopter_ID', axis=1)

# 2. Encode Categorical Features
# The target variable (Pawsonality) needs to be converted to numbers (0 to 7)
le_pawsonality = LabelEncoder()
dt_data['Target_Pawsonality_Encoded'] = le_pawsonality.fit_transform(dt_data['Target_Pawsonality'])

# Use One-Hot Encoding for the remaining categorical features (Housing_Type, Experience_Level)
X_encoded = pd.get_dummies(
    dt_data.drop('Target_Pawsonality', axis=1),
    columns=['Housing_Type', 'Experience_Level'],
    drop_first=True
)

# Separate Features (X) and Target (y)
X = X_encoded.drop('Target_Pawsonality_Encoded', axis=1)
y = X_encoded['Target_Pawsonality_Encoded']

# 3. Split Data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 4. Train the Decision Tree Model
dt_model = DecisionTreeClassifier(max_depth=7, random_state=42)
dt_model.fit(X_train, y_train)
print("Decision Tree Model training complete.")

# 5. Evaluate the Model
y_pred = dt_model.predict(X_test)

print("\n--- Model Evaluation ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("\nClassification Report (Pawsonality Types):")
# Display the report using the actual Pawsonality names
target_names = le_pawsonality.classes_
print(classification_report(y_test, y_pred, target_names=target_names))

# 6. Save the trained model and the Pawsonality encoder
joblib.dump(dt_model, 'furever_dt_pawsonality_model.pkl')
joblib.dump(le_pawsonality, 'furever_pawsonality_encoder.pkl')
print("\nDecision Tree Model and Encoder saved.")

--- Starting Decision Tree Model Training (Pawsonality) ---
Decision Tree Model training complete.

--- Model Evaluation ---
Accuracy: 0.9900

Classification Report (Pawsonality Types):
                    precision    recall  f1-score   support

 Active Adventurer       1.00      1.00      1.00        19
    Balanced Buddy       1.00      1.00      1.00        19
Confident Guardian       1.00      0.75      0.86         4
    Cozy Companion       1.00      1.00      1.00        14
   Gentle Nurturer       1.00      1.00      1.00        18
Playful Enthusiast       1.00      1.00      1.00         5
   Quiet Caretaker       0.91      1.00      0.95        10
  Social Butterfly       1.00      1.00      1.00        11

          accuracy                           0.99       100
         macro avg       0.99      0.97      0.98       100
      weighted avg       0.99      0.99      0.99       100


Decision Tree Model and Encoder saved.


**LINEAR REGRESSION**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score
import joblib

print("--- Starting Linear Regression Model Training (Adoption Predictions) ---")

# 1. Load the Prepared Data
lr_data = pd.read_csv('6_lr_adoption_prediction_training.csv')

# 2. Separate Features (X) and Target (y)
# Target is 'Target_Days_In_Shelter'
X = lr_data.drop('Target_Days_In_Shelter', axis=1)
y = lr_data['Target_Days_In_Shelter']

# 3. Split Data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 4. Train the Linear Regression Model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
print("Linear Regression Model training complete.")

# 5. Evaluate the Model
y_pred = lr_model.predict(X_test)

print("\n--- Model Evaluation ---")
# MAE: Shows the average error in days (e.g., predicted 15 days, true was 12 days, error is 3 days)
print(f"Mean Absolute Error (MAE): {mean_absolute_error(y_test, y_pred):.2f} days")
# R2 Score: Measures how well the model explains the variance (closer to 1.0 is better)
print(f"R-squared (R2) Score: {r2_score(y_test, y_pred):.4f}")

# 6. Save the trained model
joblib.dump(lr_model, 'furever_lr_adoption_model.pkl')
print("\nLinear Regression Model saved as 'furever_lr_adoption_model.pkl'.")

--- Starting Linear Regression Model Training (Adoption Predictions) ---
Linear Regression Model training complete.

--- Model Evaluation ---
Mean Absolute Error (MAE): 5.13 days
R-squared (R2) Score: 0.7649

Linear Regression Model saved as 'furever_lr_adoption_model.pkl'.


**NAIVE BAYES**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from ast import literal_eval  # Convert stringified lists back to lists
import joblib
from sklearn.metrics import accuracy_score, jaccard_score

print("--- Starting Naive Bayes Model Training (Auto-Tagging) ---")

# 1. Load the Prepared Data
nb_data = pd.read_csv('3_nb_autotagging_training.csv')

# Convert the string representation of lists in Target_Pet_Tags back into actual lists
nb_data['Target_Pet_Tags'] = nb_data['Target_Pet_Tags'].apply(literal_eval)

# 2. Separate Features (Text) and Target (Tags)
X = nb_data['Pet_Description']
y_labels = nb_data['Target_Pet_Tags']

# Optional light cleanup to remove markdown '**' used in generated text
X_clean = X.str.replace('*', '', regex=False)

# 3. Vectorize the Text Data (Feature Engineering for Text)
# Use richer features: unigrams + bigrams, larger vocab, English stopwords
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), max_features=5000)
X_vectorized = vectorizer.fit_transform(X_clean)

# 4. Binarize the Tags (Target Encoding for Multi-Label)
mlb = MultiLabelBinarizer()
y_binarized = mlb.fit_transform(y_labels)

# 5. Split Data (using the vectorized features)
X_train, X_test, y_train, y_test = train_test_split(
    X_vectorized, y_binarized, test_size=0.2, random_state=42
)

# 6. Train the Naive Bayes Model
nb_classifier = OneVsRestClassifier(MultinomialNB())
nb_classifier.fit(X_train, y_train)
print("Naive Bayes Model training complete.")

# 7. Evaluate the Model
y_pred = nb_classifier.predict(X_test)
print("\n--- Model Evaluation ---")
print(f"Subset Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Jaccard Score: {jaccard_score(y_test, y_pred, average='samples'):.4f}")

# 8. Save the trained model and vectorizers/encoders
joblib.dump(nb_classifier, 'furever_nb_autotagging_model.pkl')
joblib.dump(vectorizer, 'furever_nb_vectorizer.pkl')
joblib.dump(mlb, 'furever_nb_mlb.pkl')
print("\nNaive Bayes Model, Vectorizer, and MultiLabelBinarizer saved.")

--- Starting Naive Bayes Model Training (Auto-Tagging) ---
Naive Bayes Model training complete.

--- Model Evaluation ---
Subset Accuracy: 1.0000
Jaccard Score: 1.0000

Naive Bayes Model, Vectorizer, and MultiLabelBinarizer saved.


**KNN**

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neighbors import NearestNeighbors
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib

print("--- Starting KNN Model Training (Recommendation Engine) ---")

# 1. Load the Pet Feature Data (This data is different from the text data)
# Corrected filename
knn_data = pd.read_csv('4_knn_recommendation_features.csv')

# Use a Pet ID column to map results back to actual pets.
# We'll drop it before scaling/training, but keep a copy of the index.
# The Pet_ID is the index of the dataframe, so we reset the index to get it as a column
knn_data = knn_data.reset_index()
pet_ids = knn_data['Pet_ID']
knn_data = knn_data.drop('Pet_ID', axis=1) # Drop ID before training
# Drop the 'index' column that was created by reset_index()
knn_data = knn_data.drop('index', axis=1)


# 2. Define Feature Types (Corrected column names based on generated data)
numerical_features = ['Energy_Level']
categorical_features = [col for col in knn_data.columns if col not in numerical_features] # Dynamically determine categorical features

# 3. Create Preprocessing Pipelines
# This ensures scaling is only applied to numerical data and encoding to categorical data.
# We don't need one-hot encoding here as the categorical features are already one-hot encoded
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        # Removed 'cat' transformer as features are already encoded
    ],
    remainder='passthrough' # Keep any other columns if they exist (these are the one-hot encoded columns)
)


# 4. Apply Preprocessing
# Fit and transform the data
X_processed = preprocessor.fit_transform(knn_data)
print("Data preprocessing (Scaling of numerical features) complete.")

# 5. Train the NearestNeighbors Model
# We are not doing classification here; we are finding neighbors (similarity).
# metric='cosine' is often preferred for high-dimensional similarity.
nn_model = NearestNeighbors(n_neighbors=10, metric='cosine')
nn_model.fit(X_processed)
print(f"NearestNeighbors Model fitted on {X_processed.shape[0]} pets.")

# 6. Save the trained model and the preprocessor
joblib.dump(nn_model, 'furever_nn_recommendations_model.pkl')
joblib.dump(preprocessor, 'furever_nn_preprocessor.pkl')
joblib.dump(pet_ids, 'furever_nn_pet_ids.pkl') # Save the mapping ID list

print("\nKNN Model, Preprocessor, and Pet IDs saved for deployment.")

--- Starting KNN Model Training (Recommendation Engine) ---
Data preprocessing (Scaling of numerical features) complete.
NearestNeighbors Model fitted on 1000 pets.

KNN Model, Preprocessor, and Pet IDs saved for deployment.


**ANN**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score
import joblib

print("--- Starting ANN (MLP) Model Training (Deep Matching) ---")

# 1. Load the Deep Matching Data
# This dataset already contains one-hot (dummy) columns created earlier.
ann_data = pd.read_csv('2_5_svm_ann_matching_training.csv')

# 2. Separate Features (X) and Target (y)
X = ann_data.drop('Success_Match', axis=1)
y = ann_data['Success_Match']

# 3. Define feature types (scale only the true numeric features; pass through dummies)
numerical_features = ['Activity_Energy_Diff', 'Has_Kids', 'Good_With_Kids']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
    ],
    remainder='passthrough'  # keep the already-one-hot dummy columns as-is
)

# 4. Split Data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 5. Create Model Pipeline (Preprocessor + Classifier)
ann_classifier = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', MLPClassifier(
        hidden_layer_sizes=(100, 50),
        max_iter=300,
        random_state=42,
        solver='adam'
    ))
])

# 6. Train the Model
ann_classifier.fit(X_train, y_train)
print("ANN Deep Matching Model training complete.")

# 7. Evaluate the Model
y_pred = ann_classifier.predict(X_test)

print("\n--- ANN Model Evaluation ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred):.4f}")

# 8. Save the trained model Pipeline and original feature list
joblib.dump(ann_classifier, 'furever_ann_deepmatch_pipeline.pkl')
joblib.dump(X.columns.tolist(), 'furever_ann_feature_names.pkl')

print("\nANN Deep Match Pipeline and original feature names saved.")

--- Starting ANN (MLP) Model Training (Deep Matching) ---
ANN Deep Matching Model training complete.

--- ANN Model Evaluation ---
Accuracy: 0.9264
F1 Score: 0.8972

ANN Deep Match Pipeline and original feature names saved.


**TEST**

In [ ]:
  # --- Update the Load Components Section ---

# Find this section in your testing script and modify the NB model line:
try:
    # 1. NB Auto-Tagging
    # CHANGE THIS LINE to load the tuned model
    nb_classifier = joblib.load('furever_nb_autotagging_model.pkl')
    nb_vectorizer = joblib.load('furever_nb_vectorizer.pkl')
    nb_mlb = joblib.load('furever_nb_mlb.pkl')


    # ... (Keep the loading for KNN and ANN the same) ...

    print("✅ All model components loaded successfully for testing.")

except FileNotFoundError as e:
    print(f"❌ Error loading file: {e}. Cannot proceed.")


# --- Re-run the Failed Test ---
# Execute the entire testing cell again to re-check TEST 1: Naive Bayes Auto-Tagging

✅ All model components loaded successfully for testing.


In [ ]:
import joblib
import pandas as pd
import numpy as np  # Import numpy for the generate_description function

# --- LOAD ALL COMPONENTS ---
try:
    # 1. NB Auto-Tagging
    nb_classifier = joblib.load('furever_nb_autotagging_model.pkl')
    nb_vectorizer = joblib.load('furever_nb_vectorizer.pkl')
    nb_mlb = joblib.load('furever_nb_mlb.pkl')

    # 2. KNN Recommendations
    nn_model = joblib.load('furever_nn_recommendations_model.pkl')
    nn_preprocessor = joblib.load('furever_nn_preprocessor.pkl')
    pet_ids_list = joblib.load('furever_nn_pet_ids.pkl')

    # 3. ANN Deep Matching
    ann_pipeline = joblib.load('furever_ann_deepmatch_pipeline.pkl')
    ann_feature_names = joblib.load('furever_ann_feature_names.pkl')

    print("✅ All model components loaded successfully for testing.")

except FileNotFoundError as e:
    print(f"❌ Error loading file: {e}. Please ensure all files are saved correctly.")


# ====================================================================
# --- TEST 1: AUTO-TAGGING (NB) ---
# ====================================================================
print("\n--- TEST 1: Naive Bayes Auto-Tagging ---")

# Replicate the data generation logic for a specific test case

def generate_description_for_test(row):
    desc = f"A {row['Age_Group'].lower().replace('_', ' ')} {row['Type'].lower()} with {row['Grooming_Needs'].lower()} grooming needs. "

    # Add energy/sociability text
    if row['Energy_Level'] == 3:
        desc += "Requires lots of space and playtime, definitely **needs an active home**. "
    elif row['Energy_Level'] == 1:
        desc += "Very calm and quiet, perfect for an **apartment-approved** life. "

    # Add family/kid text
    if row['Good_With_Kids'] == 1:
        desc += "**Loves kids** and other pets. "
        tags = ['family-friendly', 'social']
    elif row['Good_With_Kids'] == 2:
        desc += "Needs a home with **experienced, older children**. "
        tags = ['experienced-owner-recommended']
    else:  # Good_With_Kids == 0
        desc += "Best suited for a **quiet, adult-only** household. "
        tags = ['adults-only']

    # Add size/other tag
    if row['Size'] == 'Large':
        desc += "Due to its size, a **house with a yard is a must**."
        tags.append('large-breed')

    # De-duplicate and return
    tags = list(dict.fromkeys(tags))
    return desc, tags

# Define a test pet profile that should generate the desired tags
test_pet_profile = pd.Series({
    'Age_Group': 'Senior',
    'Type': 'Dog',
    'Grooming_Needs': 'Low',
    'Energy_Level': 1,  # Low energy -> apartment-approved
    'Good_With_Kids': 0,  # Not good with kids -> adults-only
    'Size': 'Small'  # Doesn't add large-breed tag
})

# Generate the description and expected tags using the same logic
test_desc_1, expected_tags_1 = generate_description_for_test(test_pet_profile)

# Clean the description similarly to training cleanup
test_desc_1_clean = test_desc_1.replace('*', '')

# Use the saved vectorizer's .transform() method
X_test_nb = nb_vectorizer.transform([test_desc_1_clean])
y_pred_nb = nb_classifier.predict(X_test_nb)
predicted_tags = list(nb_mlb.inverse_transform(y_pred_nb)[0])

print(f"Input Description (Generated): '{test_desc_1}'")
print(f"Expected Tags: {expected_tags_1}")
print(f"Resulting Tags (Predicted): {predicted_tags}")

# Check if the predicted tags exactly match the expected tags
assert sorted(predicted_tags) == sorted(expected_tags_1), f"NB Test Failed: Predicted tags {predicted_tags} do not match expected tags {expected_tags_1}."
print("TEST 1 Status: Passed functional check.")


# ====================================================================
# --- TEST 2: RECOMMENDATIONS (KNN) ---
# ====================================================================
print("\n--- TEST 2: KNN Recommendations ---")

# Load the exact feature matrix used during training (indexed by Pet_ID)
knn_features_df = pd.read_csv('4_knn_recommendation_features.csv', index_col=0)

# Pick a real Pet_ID from the fitted model's order if available, otherwise take the first from the CSV
candidate_pet_id = pet_ids_list.iloc[0] if hasattr(pet_ids_list, 'iloc') and len(pet_ids_list) > 0 else knn_features_df.index[0]
if candidate_pet_id not in knn_features_df.index:
    # Fallback to a guaranteed ID
    candidate_pet_id = knn_features_df.index[0]

query_row = knn_features_df.loc[[candidate_pet_id]]  # keep as DataFrame

# Preprocess and query neighbors
X_query = nn_preprocessor.transform(query_row)
distances, indices = nn_model.kneighbors(X_query, n_neighbors=min(6, len(pet_ids_list)))

# Map indices to Pet IDs and drop the query pet itself if present
all_ids = pet_ids_list.iloc[indices[0]].tolist()
recommended_ids = [pid for pid in all_ids if pid != candidate_pet_id][:5]

print(f"Query Pet_ID: {candidate_pet_id}")
print(f"Recommended Pet IDs: {recommended_ids}")
assert len(recommended_ids) == 5, "KNN Test Failed: Did not return 5 recommendations."
print("TEST 2 Status: Passed functional check (returned expected count).")


# ====================================================================
# --- TEST 3: DEEP MATCHING (ANN) ---
# ====================================================================
print("\n--- TEST 3: ANN Deep Matching ---")

# Construct a strong match vector using the exact original training columns
# The training CSV already contains the dummy columns; we'll build a minimal row and align.

test_match_data_good_vector = pd.DataFrame({
    'Activity_Energy_Diff': [0],
    'Has_Kids': [1],
    'Good_With_Kids': [1],
    'Experience_Level_First_Time': [0],
    'Experience_Level_Past_Owner': [1],
    'Grooming_Needs_Low': [1],
    'Grooming_Needs_Medium': [0],
    'Size_Medium': [0],
    'Size_Small': [1]
})

# Align to the exact column order expected by the pipeline
for col in ann_feature_names:
    if col not in test_match_data_good_vector.columns:
        test_match_data_good_vector[col] = 0

test_df_ann_aligned = test_match_data_good_vector[ann_feature_names]

match_probability = ann_pipeline.predict_proba(test_df_ann_aligned)[:, 1][0]
match_score = int(match_probability * 100)

print(f"AI DeepMatch Score: {match_score}%")
assert match_score > 80, f"ANN Test Failed: Score {match_score}% is too low for a good match."
print("TEST 3 Status: Passed functional check (score > 80%).")

✅ All model components loaded successfully for testing.

--- TEST 1: Naive Bayes Auto-Tagging ---
Input Description (Generated): 'A senior dog with low grooming needs. Very calm and quiet, perfect for an **apartment-approved** life. Best suited for a **quiet, adult-only** household. '
Expected Tags: ['adults-only']
Resulting Tags (Predicted): ['adults-only']
TEST 1 Status: Passed functional check.

--- TEST 2: KNN Recommendations ---
Query Pet Feature Vector (Example: High Energy, Young Adult, Medium Dog, Medium Grooming)
Recommended Pet IDs: ['P0042', 'P0953', 'P0650', 'P0660', 'P0497']
TEST 2 Status: Passed functional check (returned expected count).

--- TEST 3: ANN Deep Matching ---
User/Pet Match Input (Example: Good compatibility features)
AI DeepMatch Score: 95%
TEST 3 Status: Passed functional check (score > 80%).


In [ ]:
import os
from google.colab import files
import zipfile

print("🔍 Checking for model files...")

# List of all required model files
model_files = [
    'furever_dt_pawsonality_model.pkl',
    'furever_pawsonality_encoder.pkl',
    'furever_svm_model.pkl',
    'furever_svm_scaler.pkl',
    'furever_nb_autotagging_model.pkl',
    'furever_nb_vectorizer.pkl',
    'furever_nb_mlb.pkl',
    'furever_nn_recommendations_model.pkl',
    'furever_nn_preprocessor.pkl',
    'furever_nn_pet_ids.pkl',
    'furever_ann_deepmatch_pipeline.pkl',
    'furever_ann_feature_names.pkl',
    'furever_lr_adoption_model.pkl'
]

# Check which files exist
missing_files = []
existing_files = []

for file in model_files:
    if os.path.exists(file):
        existing_files.append(file)
        print(f"✅ Found: {file}")
    else:
        missing_files.append(file)
        print(f"❌ Missing: {file}")

if missing_files:
    print(f"\n⚠️  Warning: {len(missing_files)} model file(s) missing!")
    print("Missing files:")
    for f in missing_files:
        print(f"  - {f}")
    print("\nMake sure all training cells completed successfully.")
else:
    print(f"\n✅ All {len(model_files)} model files found!")

if len(existing_files) > 0:
    print(f"\n📦 Creating furever_models.zip with {len(existing_files)} files...")

    # Create a ZIP file with all models
    with zipfile.ZipFile('furever_models.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file in existing_files:
            zipf.write(file)
            print(f"   ✓ Added: {file}")

    zip_size = os.path.getsize('furever_models.zip') / 1024
    print(f"\n✅ Created furever_models.zip")
    print(f"   Size: {zip_size:.2f} KB")

    print("\n📥 Starting download...")
    files.download('furever_models.zip')

    print("\n🎉 Download complete!")
    print("\n📋 Next steps:")
    print("1. Find 'furever_models.zip' in your Downloads folder")
    print("2. Extract the ZIP file")
    print("3. Copy all .pkl files to: server/models/")
    print("4. Run: python server.py")
else:
    print("\n❌ No model files found. Cannot create ZIP.")
    print("Run all training cells in your notebook first.")

🔍 Checking for model files...
✅ Found: furever_dt_pawsonality_model.pkl
✅ Found: furever_pawsonality_encoder.pkl
✅ Found: furever_svm_model.pkl
✅ Found: furever_svm_scaler.pkl
✅ Found: furever_nb_autotagging_model.pkl
✅ Found: furever_nb_vectorizer.pkl
✅ Found: furever_nb_mlb.pkl
✅ Found: furever_nn_recommendations_model.pkl
✅ Found: furever_nn_preprocessor.pkl
✅ Found: furever_nn_pet_ids.pkl
✅ Found: furever_ann_deepmatch_pipeline.pkl
✅ Found: furever_ann_feature_names.pkl
✅ Found: furever_lr_adoption_model.pkl

✅ All 13 model files found!

📦 Creating furever_models.zip with 13 files...
   ✓ Added: furever_dt_pawsonality_model.pkl
   ✓ Added: furever_pawsonality_encoder.pkl
   ✓ Added: furever_svm_model.pkl
   ✓ Added: furever_svm_scaler.pkl
   ✓ Added: furever_nb_autotagging_model.pkl
   ✓ Added: furever_nb_vectorizer.pkl
   ✓ Added: furever_nb_mlb.pkl
   ✓ Added: furever_nn_recommendations_model.pkl
   ✓ Added: furever_nn_preprocessor.pkl
   ✓ Added: furever_nn_pet_ids.pkl
   ✓ Adde

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


🎉 Download complete!

📋 Next steps:
1. Find 'furever_models.zip' in your Downloads folder
2. Extract the ZIP file
3. Copy all .pkl files to: server/models/
4. Run: python server.py
